#### This notebook is intended for basic GEE stuff. It was the first one I created and it was based at first on the S1 change detection guide

Kernel: home_town

In [ ]:
import ee
import IPython.display as disp
import folium
from IPython.display import display, Image
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import norm, gamma, f, chi2


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

#### single image processing

In [ ]:
#Vindefjallen
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              16.089696198355398,
              66.09725541788251
            ],
            [
              16.026316207001685,
              66.09725541788251
            ],
            [
              16.026316207001685,
              66.06875216004511
            ],
            [
              16.089696198355398,
              66.06875216004511
            ],
            [
              16.089696198355398,
              66.09725541788251
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [ ]:
# ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
#                        .filterBounds(aoi) 
#                        .filterDate(ee.Date('2022-01-01'),ee.Date('2022-02-28')) 
#                        .first() 
#                        .clip(aoi))
# print(ffa_db.bandNames().getInfo())
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))
print(ffa_fl.bandNames().getInfo())

In [ ]:
# #get orbit number   
# orbitproperties = ffa_db.get('orbitProperties_pass').getInfo()
# relativeOrbitNumber = ffa_db.get('relativeOrbitNumber_start').getInfo()
# #print both
# print(orbitproperties)
# print(relativeOrbitNumber)

orbitproperties = ffa_fl.get('orbitProperties_pass').getInfo()
relativeOrbitNumber = ffa_fl.get('relativeOrbitNumber_start').getInfo()
#print both
print(orbitproperties)
print(relativeOrbitNumber)

In [ ]:
# url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
# disp.Image(url=url, width=800)

url = ffa_fl.select('VH').getThumbURL({'min': 0, 'max': .1})
disp.Image(url=url, width=800)

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_fl.select('VV'),
                   ffa_fl.select('VH'),
                   ffa_fl.select('VV').divide(ffa_fl.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [0, 0,0], 'max': [.2, .2, .2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

#### Multi temporal

In [ ]:
#Vindefjallen area, with 2 measurements. shp is in arcigs
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "id": 1,
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              16.042947830545454,
              65.90158276444853
            ],
            [
              16.025595339138334,
              65.9014970392025
            ],
            [
              16.008242949326043,
              65.90140935272971
            ],
            [
              16.00871237738704,
              65.88605107102144
            ],
            [
              16.026054388085974,
              65.88613871942898
            ],
            [
              16.04339650016763,
              65.88622440801343
            ],
            [
              16.042947830545454,
              65.90158276444853
            ]
          ]
        ]
      },
      "properties": {
        "OBJECTID": 1,
        "Shape_Length": 6586.801446635423,
        "Shape_Area": 2707353.895849061
      }
    }
  ]
}
coords_mini = geoJSON['features'][0]['geometry']['coordinates']
aoi_mini = ee.Geometry.Polygon(coords_mini)

In [ ]:
#náð í orbit information fyrir næsta skref þar sem náð er í myndir	
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi_mini) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi_mini))
orbitproperties = ffa_fl.get('orbitProperties_pass').getInfo()
relativeOrbitNumber = ffa_fl.get('relativeOrbitNumber_start').getInfo()
#print both
print(orbitproperties)
print(relativeOrbitNumber)

In [ ]:
#multi temporal
#Náð í myndir fyrir tímabil og orbit frá GEE. Hægt að velja dB eða float
def apply_for_images(type):

    date_1 = ee.Date('2022-01-01')
    date_2 = ee.Date('2023-01-01')  

    if type == 'dB':
        typex = 'COPERNICUS/S1_GRD' 
    elif type == 'float':
        typex = 'COPERNICUS/S1_GRD_FLOAT' 

    im_coll = (ee.ImageCollection(typex)
        .filterBounds(aoi_mini)
        .filterDate(date_1,date_2)
        .filter(ee.Filter.eq('orbitProperties_pass', orbitproperties))
        .filter(ee.Filter.eq('relativeOrbitNumber_start', relativeOrbitNumber))
        .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
        .sort('date'))
        
    timestamplist = (im_coll.aggregate_array('date')
                    .map(lambda d: ee.String('T').cat(ee.String(d)))
                    .getInfo())
    
    return im_coll, timestamplist

In [ ]:
#get mean value for im1
def gee_stats(gee_area, pol):
    """Get stat values for image"""
    mean = gee_area.reduceRegion(ee.Reducer.mean(),aoi_mini,10).get(pol).getInfo()
    #and get min and max values
    min = gee_area.reduceRegion(ee.Reducer.min(),aoi_mini,10).get(pol).getInfo()
    max = gee_area.reduceRegion(ee.Reducer.max(),aoi_mini,10).get(pol).getInfo()
    #standard deviation
    std = gee_area.reduceRegion(ee.Reducer.stdDev(),aoi_mini,10).get(pol).getInfo()
    #print as f string with new line
    print()
    print(f'mean: {mean}\n min: {min}\nmax: {max}\nstd: {std}')
    return (mean, std)

In [ ]:
#Náð í S1 multi temporal myndir fyrir tímabil og orbit frá GEE. Hægt að velja dB eða float
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi_mini)

#float
imgCollecting_float, timestamp_list = apply_for_images('float') #náð í myndir fyrir tímabil og orbit frá GEE
image_list_float = imgCollecting_float.toList(imgCollecting_float.size()) #breyta í GEE lista
image_list_float = ee.List(image_list_float.map(clip_img)) #clippa myndir eftir AOI

im0_fl = ee.Image(image_list_float.get(0)).select('VV')
mean, std = gee_stats(im0_fl, 'VV')
url1 = im0_fl.getThumbURL({'min': 0, 'max': mean + (std*2)})
display(Image(url=url1, width=600))

# #DB
# image_list_dB = apply_for_images('dB')[0]
# image_list_dB = image_list_dB.toList(image_list_dB.size())
# image_list_dB = ee.List(image_list_dB.map(clip_img))

# im0_db = ee.Image(image_list_dB.get(0)).select('VV')
# mean, std = gee_stats(im0_db, 'VV')
# url2 = im0_db.getThumbURL({'min': mean - (std*2), 'max': mean + (std*2)})
# display(Image(url=url2, width=600))

In [26]:
#Diplay entire collection
collection_size = image_list_float.size().getInfo()

for i in range(0, collection_size):
    im1 = ee.Image(image_list_float.get(i)).select('VV')
    url = im1.getThumbURL({'min': 0, 'max': mean + (std*2)})
    print(timestamp_list[i])
    display(Image(url=url, width=600))

T20220101


T20220113


T20220125


T20220206


T20220218


...

T20221109


T20221121


T20221203


T20221215


T20221227


###

In [ ]:
#for arcgis
def convert_to_datetime(date_string):
    # Extract year, month, and day from the date string
    year = int(date_string[1:5])
    month = int(date_string[5:7])
    day = int(date_string[7:9])

    # Create a datetime object
    return datetime(year, month, day)

# Convert each date string to a datetime object
date_objects = [convert_to_datetime(date_str) for date_str in timestamp_list]
for i in date_objects:
    print(i.strftime('%d/%m/%Y'))

In [ ]:

def convert_to_datetime(date_string):
    # Extract year, month, and day from the date string
    year = int(date_string[1:5])
    month = int(date_string[5:7])
    day = int(date_string[7:9])

    # Create a datetime object
    return datetime(year, month, day)

# Convert each date string to a datetime object
date_objects = [convert_to_datetime(date_str) for date_str in timestamp_list]
timestamp_list_modified = []
for date_obj in date_objects:
    # print(date_obj.strftime('%Y-%m-%d'))
    print(date_obj.strftime('%d/%m/%Y'))
    timestamp_list_modified.append(date_obj.strftime('%Y-%m-%d'))

print(timestamp_list_modified)

print(timestamp_list)


In [ ]:
#testing
# Define a function for exporting an image
def export_image_to_drive(image, description, folder, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region,
        fileNamePrefix=description,
        crs='EPSG:4326',
        fileFormat='GeoTIFF'
    )
    task.start()
collection_size = imgCollecting_float.size().getInfo()

# Iterate over the collection using iterate
for i in range(collection_size):
    # Get the ith image in the collection
    image = ee.Image(imgCollecting_float.toList(collection_size).get(i))
    image_sel = image.select('VV')
    export_image_to_drive(
        image=image_sel,
        description=f'testing2_{i}',
        folder='my_exports',
        scale=30,
        region=aoi_mini
    )
    # Do something with the image (e.g., print image ID)
    image_id = image.id().getInfo()
    print(f"Image {i + 1} ID: {image_id}")
# Iterate over the list and export each image
# for i, image in enumerate(image_list):
#     export_image_to_drive(
#         image=image,
#         description=f'my_export_lyon_{i}',
#         folder='my_exports',
#         scale=30,
#         region=aoi_mini
#     )


In [ ]:
#testing!
# task.status()
def export_image_to_drive(image, description, folder, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region,
        fileNamePrefix=description,
        crs='EPSG:4326',
        fileFormat='GeoTIFF'
    )
    task.start()
collection_size = imgCollecting_float.size().getInfo()

for i, date in enumerate(timestamp_list_modified):
    print(f"Image {i + 1} ID: {date}")
        # Get the ith image in the collection
    image = ee.Image(imgCollecting_float.toList(collection_size).get(i))
    image_sel = image.select('VV')
    export_image_to_drive(
        image=image_sel,
        description=f'Vindefjallen_AOI_VV_{i}_{date}',
        folder='my_exports',
        scale=30,
        region=aoi_mini
    )

In [ ]:
task.status()

In [ ]:
#calculate ratio of an entire image
imVV = ee.Image(image_list_dB.get(4)).select('VV')
imVH  = ee.Image(image_list_dB.get(4)).select('VH')





ratio = imVH.divide(imVV)
# url = ratio.getThumbURL({'min': -5, 'max': 5})
# disp.Image(url=url, width=600)
gee_stats(ratio, 'VH')

In [ ]:
#Display rgb image on folium map
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
  folium.raster_layers.TileLayer( #Add ESRI basemap
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
location = aoi_mini.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
# rgb = ee.Image.rgb(image.select('VV'),
#                    image.select('VH'),
#                    image.select('VV').divide(image.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=12)


# Add the S1 rgb composite to the map object.
m.add_ee_layer(p_value, {'min': [0], 'max': [3]}, 'Ratio - VH/VV')
# m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'RGB - VV,VH,VV/VH')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [ ]:
#create a function to calculate the ratio
def get_avg_ratio(img):

    ratio = ee.Image(img).select('VH').divide(ee.Image(img).select('VV'))
    mean = ratio.reduceRegion(
        ee.Reducer.mean()).getInfo()
    return mean
get_avg_ratio(im_list.get(2))
#create function to calculate the mean
def get_avg(img):
    mean = ee.Image(img).reduceRegion(
        ee.Reducer.mean()).getInfo()
    return mean

In [ ]:
#get date of multi image list
import time

acq_times = im_coll.aggregate_array('system:time_start').getInfo()
time_list = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
time_list

### Histograms and other stuff

https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/detecting-changes-in-sentinel-1-imagery-pt-1/index.ipynb#scrollTo=KfjUo-FyUklG

In [ ]:
pol = 'VV'

ffa_fl = ee.Image(image_list_float.get(0)).select(pol)


hist = ffa_fl.select(pol).reduceRegion(
    ee.Reducer.fixedHistogram(0, 0.5, 500),aoi_mini).get(pol).getInfo()
mean = ffa_fl.select(pol).reduceRegion(
    ee.Reducer.mean(), aoi_mini).get(pol).getInfo()
variance = ffa_fl.select(pol).reduceRegion(
    ee.Reducer.variance(), aoi_mini).get(pol).getInfo()

a = np.array(hist)
x = a[:, 0]                 # array of bucket edge positions
y = a[:, 1]/np.sum(a[:, 1]) # normalized array of bucket contents
plt.grid()
plt.plot(x, y, '.')
plt.show()

alpha = 4
beta = mean/alpha
plt.grid()
plt.plot(x, y, '.', label='data')
plt.plot(x, gamma.pdf(x, alpha, 0, beta)/1000, '-r', label='gamma')
plt.legend()
plt.show()

In [ ]:
from pprint import pprint

# A Lansat 8 TOA image collection for a specific year and location.
col = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterBounds(
    ee.Geometry.Point([-122.073, 37.188])).filterDate('2018', '2019')

# An image property of interest, percent cloud cover in this case.
prop = 'CLOUD_COVER'

# Use ee.ImageCollection.aggregate_* functions to fetch information about
# values of a selected property across all images in the collection. For
# example, produce a list of all values, get counts, and calculate statistics.
print('List of property values:', col.aggregate_array(prop).getInfo())
print('Count of property values:', col.aggregate_count(prop).getInfo())
print('Count of distinct property values:',
      col.aggregate_count_distinct(prop).getInfo())
print('First collection element property value:',
      col.aggregate_first(prop).getInfo())
print('Histogram of property values:')
pprint(col.aggregate_histogram(prop).getInfo())
print('Min of property values:', col.aggregate_min(prop).getInfo())
print('Max of property values:', col.aggregate_max(prop).getInfo())

# The following methods are applicable to numerical properties only.
print('Mean of property values:', col.aggregate_mean(prop).getInfo())
print('Sum of property values:', col.aggregate_sum(prop).getInfo())
print('Product of property values:', col.aggregate_product(prop).getInfo())
print('Std dev (sample) of property values:',
      col.aggregate_sample_sd(prop).getInfo())
print('Variance (sample) of property values:',
      col.aggregate_sample_var(prop).getInfo())
print('Std dev (total) of property values:',
      col.aggregate_total_sd(prop).getInfo())
print('Variance (total) of property values:',
      col.aggregate_total_var(prop).getInfo())
print('Summary stats of property values:')
pprint(col.aggregate_stats(prop).getInfo())

# Note that if the property is formatted as a string, min and max will
# respectively return the first and last values according to alphanumeric
# order of the property values.
prop_string = 'LANDSAT_SCENE_ID'
print('List of property values (string):',
      col.aggregate_array(prop_string).getInfo())
print('Min of property values (string):',
      col.aggregate_min(prop_string).getInfo())
print('Max of property values (string):',
      col.aggregate_max(prop_string).getInfo())

### Annað elda stuff

In [ ]:
#get list length
len_list = im_list.length().getInfo()
for i in range(len_list):
    val = get_avg_ratio(im_list.get(i))
    value_ratio = val.get('VH')
    value_VV = get_avg(ee.Image(im_list.get(i)).select('VV'))
    value_VH = get_avg(ee.Image(im_list.get(i)).select('VH'))
    #print values with ; inbewteen
    print(time_list[i], value_ratio, value_VV.get('VV'), value_VH.get('VH'), sep=';')

In [ ]:
url = ratio.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)


In [ ]:
hist = ratio.reduceRegion(ee.Reducer.fixedHistogram(0, 5, 500), aoi).get('VV').getInfo()
mean = ratio.reduceRegion(ee.Reducer.mean(), aoi).get('VV').getInfo()
variance = ratio.reduceRegion(ee.Reducer.variance(), aoi).get('VV').getInfo()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a = np.array(hist)
x = a[:, 0]
y = a[:, 1] / np.sum(a[:, 1])
plt.grid()
plt.plot(x, y, '.')
plt.show()
